In [2]:
# coding: utf-8

# In[2]:

import pandas as pd
import time
from bs4 import BeautifulSoup
import sys, io
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.proxy import *

# @author Ranjeet Singh <ranjeetsingh867@gmail.com>
# Modify it according to your requirements

no_of_reviews = 1000

non_bmp_map = dict.fromkeys(range(0x10000, sys.maxunicode + 1), 0xfffd)
driver = webdriver.Chrome(r"C:\Users\kharr\Anaconda3\Scripts\chromedriver.exe") #這裡改成自己driver的目錄

wait = WebDriverWait( driver, 10 )


# Append your app store urls here
urls = ["https://play.google.com/store/apps/details?id=com.ubercab.eats",
       "https://play.google.com/store/apps/details?id=com.deliveroo.orderapp"] #這裡改成APP網址

counter = 1

for url in urls:

    driver.get(url)

    page = driver.page_source

    soup_expatistan = BeautifulSoup(page, "html.parser")

    expatistan_table = soup_expatistan.find("h1", class_="AHFaub")

    print("App name: ", expatistan_table.string)

    expatistan_table = soup_expatistan.findAll("span", class_="htlgb")[4]

    print("Installs Range: ", expatistan_table.string)

    expatistan_table = soup_expatistan.find("meta", itemprop="ratingValue")

    print("Rating Value: ", expatistan_table['content'])

    expatistan_table = soup_expatistan.find("meta", itemprop="reviewCount")

    print("Reviews Count: ", expatistan_table['content'])

    soup_histogram = soup_expatistan.find("div", class_="VEF2C")

    rating_bars = soup_histogram.find_all('div', class_="mMF0fd")

    for rating_bar in rating_bars:
        print("Rating: ", rating_bar.find("span").text)
        print("Rating count: ", rating_bar.find("span", class_="L2o20d").get('title'))

    # open all reviews
    url = url+'&showAllReviews=true'
    driver.get(url)
    time.sleep(5) # wait dom ready
    for i in range(1,1000): #這裡改range(1,n)，n越大程式碼跑比較久且抓比較多資料。n=50大概可以抓兩千筆資料左右。
        try:
            driver.execute_script("document.evaluate(\"//*[contains(text(), '顯示更多內容')]\", document, null, XPathResult.FIRST_ORDERED_NODE_TYPE, null).singleNodeValue.click();")
        except:
            pass
        finally:
            driver.execute_script('window.scrollTo(0, document.body.scrollHeight);') #scroll to load other reviews
            #driver.execute_script("document.getElementById('show-more-button').click();")
            time.sleep(1)
    page = driver.page_source

    soup_expatistan = BeautifulSoup(page, "html.parser")
    expand_pages = soup_expatistan.findAll("div", class_="d15Mdf")
    
    author_names = []
    review_dates = []
    reviewer_ratings = []
    review_body = []
    developer_reply = []
    review_likes = []


    for expand_page in expand_pages:

        author_names.append(str(expand_page.find("span",class_="X43Kjb").text))

        review_dates.append(expand_page.find("span", class_="p2TkOb").text)

        rating = expand_page.find("div", class_="pf5lIe").find_next()['aria-label'];
        rating = rating.split('(')[0]
        rating = ''.join(x for x in rating if x.isdigit())
        reviewer_ratings.append(rating)

        review_body.append(str(expand_page.find("div", class_="UD7Dzf").text))

        reply = expand_page.find_parent().find("div", class_="LVQB0b")
        if hasattr(reply, "text"):
            developer_reply.append(str(reply.text))
        else:
            developer_reply.append("")

        review_likes.append(expand_page.find("div",class_="jUL89d").text)   

    data = list(zip(author_names,review_dates,reviewer_ratings,review_body,developer_reply,review_likes))
    df = pd.DataFrame(data, columns = ['Author Name','Review Date','Reviewer Ratings','Review Body','Developer Reply', 'Review Likes'])
    df.index += 1
    df.to_csv(str(counter) + 'a.csv')
    #輸出csv檔，跟這個.ipynb檔目錄相同
    counter += 1
    
driver.quit()

App name:  Uber Eats：美食外送
Installs Range:  50,000,000+
Rating Value:  4.222044944763184
Reviews Count:  1224850
Rating:  5
Rating count:  None
Rating:  4
Rating count:  None
Rating:  3
Rating count:  None
Rating:  2
Rating count:  None
Rating:  1
Rating count:  None
App name:  Deliveroo 戶戶送：美食送餐服務
Installs Range:  5,000,000+
Rating Value:  4.158066749572754
Reviews Count:  83781
Rating:  5
Rating count:  None
Rating:  4
Rating count:  None
Rating:  3
Rating count:  None
Rating:  2
Rating count:  None
Rating:  1
Rating count:  None
